# Nasa Page Count

In [ ]:
import sys
sys.path.append("..")
from helpers.path_translation import translate_to_file_string
from pyspark.sql import SparkSession
import re

In [ ]:
input_file = translate_to_file_string("../../data/nasa_http_log_1995.csv")

In [ ]:
# create a SparkSession without local master and app name
spark = (SparkSession
       .builder 
       .master("local[*]") 
       .appName("Nasa Page Count")
       .getOrCreate())
# change log level to ERROR
spark.sparkContext.setLogLevel("ERROR")
# read file 
input = spark.sparkContext.textFile(input_file)
# regular expressions to find html get requests
filter_pattern = re.compile(".*GET,/.*\.html")
match_pattern = re.compile('GET,/.*\.html')
# select only lines with get Requests
filtered = input.filter(lambda s: filter_pattern.match(s)) 
# search for the match_pattern in each line and return the matching part as a key/value pair e.g. ('/ksc.html', 1) 
pages = filtered.map(lambda s: (match_pattern.search(s).group()[4:], 1))
# sum up the entries with the same key
popularity = pages.reduceByKey(lambda a, b: a+b)

In [ ]:
# show the result
print(popularity.collect())

In [25]:
spark.stop()